In [2]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from queue import PriorityQueue

In [3]:
PUZZLE_DIM = 5 
RANDOMIZE_STEPS = 100

def available_actions(state: np.ndarray) -> list:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(((x, y), (x - 1, y))) 
    if x < PUZZLE_DIM - 1:
        actions.append(((x, y), (x + 1, y)))  
    if y > 0:
        actions.append(((x, y), (x, y - 1))) 
    if y < PUZZLE_DIM - 1:
        actions.append(((x, y), (x, y + 1))) 
    return actions

def do_action(state: np.ndarray, action: tuple) -> np.ndarray:
    new_state = state.copy()
    (x1, y1), (x2, y2) = action
    new_state[x1, y1], new_state[x2, y2] = new_state[x2, y2], new_state[x1, y1]
    return new_state

def optimized_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    manhattan_distance = 0
    misplacement_count = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                manhattan_distance += abs(x - goal_x) + abs(y - goal_y)
                if goal[x, y] != value:
                    misplacement_count += 1

    return manhattan_distance + 2 * misplacement_count  

def solve_puzzle(start_state: np.ndarray, goal_state: np.ndarray):
    frontier = PriorityQueue()
    frontier.put((0, start_state.tolist(), [start_state.tolist()], 0)) 
    visited = set()

    while not frontier.empty():
        f_cost, current_state, path, g_cost = frontier.get()
        current_state = np.array(current_state)
        if np.array_equal(current_state, goal_state):
            return path  

        state_tuple = tuple(map(tuple, current_state))
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_g_cost = g_cost + 1
            new_f_cost = new_g_cost + optimized_heuristic(new_state, goal_state)
            new_path = path + [new_state.tolist()]
            frontier.put((new_f_cost, new_state.tolist(), new_path, new_g_cost))

    return None  


In [4]:
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
   
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
solution = solve_puzzle(state, goal_state)

if solution:
    print(f"Solution found in {len(solution) - 1} moves!") 
    for step, s in enumerate(solution):
        print(f"Step {step}:")
        print(np.array(s))  
else:
    print("No solution found")

Randomizing:   0%|          | 0/100 [00:00<?, ?it/s]

Solution found in 42 moves!
Step 0:
[[ 3 12  7  4  5]
 [ 6 14  8 10 15]
 [11  1  9 18  0]
 [16 17 24 19  2]
 [21 22 13 23 20]]
Step 1:
[[ 3 12  7  4  5]
 [ 6 14  8 10 15]
 [11  1  9 18  2]
 [16 17 24 19  0]
 [21 22 13 23 20]]
Step 2:
[[ 3 12  7  4  5]
 [ 6 14  8 10 15]
 [11  1  9 18  2]
 [16 17 24  0 19]
 [21 22 13 23 20]]
Step 3:
[[ 3 12  7  4  5]
 [ 6 14  8 10 15]
 [11  1  9  0  2]
 [16 17 24 18 19]
 [21 22 13 23 20]]
Step 4:
[[ 3 12  7  4  5]
 [ 6 14  8 10 15]
 [11  1  9  2  0]
 [16 17 24 18 19]
 [21 22 13 23 20]]
Step 5:
[[ 3 12  7  4  5]
 [ 6 14  8 10  0]
 [11  1  9  2 15]
 [16 17 24 18 19]
 [21 22 13 23 20]]
Step 6:
[[ 3 12  7  4  5]
 [ 6 14  8  0 10]
 [11  1  9  2 15]
 [16 17 24 18 19]
 [21 22 13 23 20]]
Step 7:
[[ 3 12  7  4  5]
 [ 6 14  8  2 10]
 [11  1  9  0 15]
 [16 17 24 18 19]
 [21 22 13 23 20]]
Step 8:
[[ 3 12  7  4  5]
 [ 6 14  8  2 10]
 [11  1  9 18 15]
 [16 17 24  0 19]
 [21 22 13 23 20]]
Step 9:
[[ 3 12  7  4  5]
 [ 6 14  8  2 10]
 [11  1  9 18 15]
 [16 17  0 24 19]
 

## Enhanced version

In [ ]:
#PUZZLE_DIM = 3  
DEFAULT_RANDOMIZE_STEPS = 100  

def available_actions(state: np.ndarray) -> list:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(((x, y), (x - 1, y)))  
    if x < PUZZLE_DIM - 1:
        actions.append(((x, y), (x + 1, y)))  
    if y > 0:
        actions.append(((x, y), (x, y - 1)))  
    if y < PUZZLE_DIM - 1:
        actions.append(((x, y), (x, y + 1)))  
    return actions

def do_action(state: np.ndarray, action: tuple) -> np.ndarray:
    new_state = state.copy()
    (x1, y1), (x2, y2) = action
    new_state[x1, y1], new_state[x2, y2] = new_state[x2, y2], new_state[x1, y1]
    return new_state

def optimized_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    manhattan_distance = 0
    misplacement_count = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                manhattan_distance += abs(x - goal_x) + abs(y - goal_y)
                if goal[x, y] != value:
                    misplacement_count += 1
    return manhattan_distance + 2 * misplacement_count  

def solve_puzzle(start_state: np.ndarray, goal_state: np.ndarray):
    frontier = PriorityQueue()
    frontier.put((0, start_state.tolist(), [start_state.tolist()], 0)) 
    visited = set()

    while not frontier.empty():
        f_cost, current_state, path, g_cost = frontier.get()
        current_state = np.array(current_state)
        if np.array_equal(current_state, goal_state):
            return path  

        state_tuple = tuple(map(tuple, current_state))
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_g_cost = g_cost + 1
            new_f_cost = new_g_cost + optimized_heuristic(new_state, goal_state)
            new_path = path + [new_state.tolist()]
            frontier.put((new_f_cost, new_state.tolist(), new_path, new_g_cost))

    return None  

def generate_randomized_state(puzzle_dim, random_steps):
    """ Generate a solvable randomized puzzle state by applying a given number of random moves. """
    state = np.array([i for i in range(1, puzzle_dim**2)] + [0]).reshape((puzzle_dim, puzzle_dim))
    
    for _ in tqdm(range(random_steps), desc=f'Randomizing ({random_steps} steps)'):
        state = do_action(state, choice(available_actions(state)))

    return state

# Allow user to specify different levels of randomization
random_steps_list = [100, 250, 500]  

for steps in random_steps_list:
    print(f"\nTesting with {steps} randomization steps:")
    
    state = generate_randomized_state(PUZZLE_DIM, steps)
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    
    solution = solve_puzzle(state, goal_state)

    if solution:
        print(f"Solution found in {len(solution) - 1} moves!\n")
    else:
        print("No solution found.\n")


Testing with 100 randomization steps:


Randomizing (100 steps):   0%|          | 0/100 [00:00<?, ?it/s]

Solution found in 30 moves!


Testing with 250 randomization steps:


Randomizing (250 steps):   0%|          | 0/250 [00:00<?, ?it/s]